In [5]:
import tensorflow as tf
import numpy as np
import datetime
import math
%autoindent

ERROR:root:Line magic function `%autoindent` not found.


In [6]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x,*args,**kwargs ):
        return x

In [7]:
filename = 'weather.npz'
data = np.load(filename)
daily = data['daily']
weekly = data['weekly']

num_weeks = len(weekly)
dates  = np.array([datetime.datetime.strptime(str(int(d)),'%Y%m%d') for d in weekly[:,0]])

In [9]:
def assign_season(date):
    ''' Assign season based on meteorological season.
        Spring - from Mar 1 to May 31
        Summer - from Jun 1 to Aug 31
        Autumn - from Sep 1 to Nov 30
        Winter - from Dec 1 to Feb 28 (Feb 29 in a leap year)
    '''
    month = date.month
    # spring = 0
    if 3 <= month < 6:
        season = 0
    # summer = 1
    elif 6 <= month < 9:
        season = 1
    # autumn = 2
    elif 9 <= month < 12:
        season = 2
    # winter = 3
    elif month == 12 or month < 3:
        season = 3
    return season

In [10]:
num_classes = 4
num_inputs = 5 
state_size = 11 

In [11]:
labels  = np.zeros([num_weeks,num_classes])
for i,d in enumerate(dates):
    labels[i,assign_season(d)] = 1

In [12]:
train = weekly[:,1:]
train = train - np.average(train,axis=0)
train = train / train.std(axis=0)


In [13]:
sess = tf.InteractiveSession()

In [14]:
x = tf.placeholder("float",[None,num_inputs])
x_ = tf.reshape(x, [1,num_weeks,num_inputs])
 

In [15]:
y_ =tf.placeholder("float", [None,num_classes])

In [18]:
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
outputs,states = tf.nn.dynamic_rnn(cell,x_,dtype=tf.nn.dtypes.float32,initial_state=None)

In [19]:
W1 = tf.Variable(tf.truncated_normal([state_size,num_classes],stddev=1./math.sqrt(num_inputs)))
b1 = tf.Variable(tf.constant(0.1,shape=[num_classes]))

In [20]:
h1 = tf.reshape(outputs,[-1,state_size])

In [24]:
sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(h1,W1)+b1)

In [25]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y + 1e-50, y_))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)


In [28]:
correct_prediction= tf.equal(tf.argmax(y_,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

In [29]:
epochs = 100
train_acc = np.zeros(epochs//10)
for i in tqdm(range(epochs), ascii=True):
    if i % 10 == 0:  # Record summary data, and the accuracy
        # Check accuracy on train set
        A = accuracy.eval(feed_dict={x: train, y_: labels})
        train_acc[i//10] = A
    train_step.run(feed_dict={x: train, y_: labels})

In [ ]:
import matplotlib.pyplot as plt
plt.ion()
plt.figure(figsize=(6, 6))
plt.plot(train_acc